In [1]:
import pandas as pd

In [4]:
file_location = "../data/task-3/rawdata.xlsx"
df = pd.read_excel(file_location)
print(df.head())

        date      time  sensor location  number activity position location.1
0 2024-01-16  10:10:30     0.0       A1       1   placed   inside         A1
1 2024-01-16  10:12:30     1.0       A2       1   picked   inside         A2
2 2024-01-16  10:28:30     1.0       A3       1   picked   Inside         A3
3 2024-01-16  10:38:20     0.0       A4       1   placed   Inside         A4
4 2024-01-16  10:45:20     1.0       A5       1   placed   inside         A5


In [7]:
# make a function that gives the number of picked and placed activites for each date
def get_picked_and_placed(df):
    picked = df[df['activity'] == 'picked'].groupby('date').size()
    placed = df[df['activity'] == 'placed'].groupby('date').size()
    return pd.concat([picked, placed], axis=1).fillna(0)

In [33]:
print(get_picked_and_placed(df))

             0   1
date              
2024-01-16  40  40
2024-01-17  10   9
2024-01-18  37  39


In [15]:
# make a function to convert time column to string
def convert_time_to_string(df):
    return df.assign(time=df['time'].astype(str))

In [21]:
df["time"] = df["time"].astype(str)

In [30]:
# convert time column to 24h format
def convert_time_to_24h(df):
    return df.assign(time=pd.to_datetime(df['time'], format='%I:%M:%S %p').dt.strftime('%H:%M:%S'))

# what do the values in format mean?
# %I - hour (12-hour clock) as a zero-padded decimal number
# %M - minute as a zero-padded decimal number
# %p - AM or PM

# is ther eone for second too?
# %S - second as a zero-padded decimal number

In [31]:
df.head()

,date,time,sensor,location,number,activity,position,location.1
0,2024-01-16,10:10:30,0.0,A1,1,placed,inside,A1
1,2024-01-16,10:12:30,1.0,A2,1,picked,inside,A2
2,2024-01-16,10:28:30,1.0,A3,1,picked,Inside,A3
3,2024-01-16,10:38:20,0.0,A4,1,placed,Inside,A4
4,2024-01-16,10:45:20,1.0,A5,1,placed,inside,A5


In [32]:
df.tail()

,date,time,sensor,location,number,activity,position,location.1
170,2024-01-18,12:45:05,0.0,A8,2,placed,inside,A8
171,2024-01-18,12:53:10,1.0,A9,2,picked,inside,A9
172,2024-01-18,12:59:10,1.0,A10,2,placed,outside,A10
173,2024-01-18,13:10:20,1.0,A11,2,picked,Inside,A11
174,2024-01-18,13:18:30,0.0,A12,2,placed,inside,A12


In [40]:
# how do i find duration for inside and outside activity each date?
def get_duration(df):
    df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S')
    df = df.sort_values(by=['date', 'time'])
    df['duration'] = df.groupby(['date', 'activity'])['time'].diff().dt.total_seconds().fillna(0)
    return df

print(get_duration(df))



          date                time  sensor location  number activity position  \
16  2024-01-16 1900-01-01 00:02:20     1.0      a17       1   placed   inside   
24  2024-01-16 1900-01-01 00:02:20     0.0      a25       1   placed   inside   
18  2024-01-16 1900-01-01 00:08:20     1.0      a19       1   picked   inside   
19  2024-01-16 1900-01-01 00:10:20     0.0      a20       1   placed   inside   
20  2024-01-16 1900-01-01 00:15:20     0.0      a21       1   placed   inside   
..         ...                 ...     ...      ...     ...      ...      ...   
149 2024-01-18 1900-01-01 18:57:00     0.0      a30       1   picked  outside   
150 2024-01-18 1900-01-01 19:10:00     NaN      a31       1   placed   Inside   
151 2024-01-18 1900-01-01 19:19:00     1.0      a32       1   picked   inside   
152 2024-01-18 1900-01-01 19:23:00     1.0      a33       1   placed   inside   
153 2024-01-18 1900-01-01 19:30:20     1.0      a34       1   picked   inside   

    location.1  duration  


In [42]:
# now sum all the inside and outside on each date
def get_sum_duration(df):
    return df.groupby(['date', 'activity'])['duration'].sum().unstack().fillna(0)

print(get_sum_duration(get_duration(df)))

activity     picked   placed
date                        
2024-01-16  42240.0  43020.0
2024-01-17  11330.0  10910.0
2024-01-18  33470.0  33150.0


In [43]:
# now combine both picked_and_placed dataframe and sum_duration dataframe
def combine_dfs(df):
    return pd.concat([get_picked_and_placed(df), get_sum_duration(get_duration(df))], axis=1).fillna(0)

print(combine_dfs(df))

             0   1   picked   placed
date                                
2024-01-16  40  40  42240.0  43020.0
2024-01-17  10   9  11330.0  10910.0
2024-01-18  37  39  33470.0  33150.0


In [44]:
# export it as csv
combine_dfs(df).to_csv('processed_data.csv')